# 0 环境配置

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2018)

from keras.optimizers import *
from model.coremodel import coremodel
from dataload.my_image import MyImageDataGenerator
from loss import myloss
from metric import diff_accuracy, count_accuracy
from keras.utils import multi_gpu_model
print("import done")

Using TensorFlow backend.


import done


# 1 预设参数

In [11]:
basedir = "/home/ubuntu/JupyterNotebook/columncount/data"
basemodel_name = 'inception_v3'
model_image_size = (378, 504)
modelname = basemodel_name + 'model_s0_20180124'

# 2 数据生成

In [3]:
# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError

In [4]:
train_batch_size = 48
valid_batch_size = 2

In [5]:
train_gen = MyImageDataGenerator(
    preprocessing_function=preprocess_input,
    channel_shift_range=50.,
)

gen = MyImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.myflow_from_directory(os.path.join(basedir, 'train'),  model_image_size,
                                                shuffle=True, batch_size=train_batch_size)

valid_generator = gen.myflow_from_directory(os.path.join(basedir, 'valid'),  model_image_size,
                                          shuffle=True, batch_size=valid_batch_size)

Found 1691 images.
Found 188 images.


In [6]:
train_samples_epoch = train_generator.samples
print("samples_train_epoch = {}".format(train_samples_epoch))

steps_train = train_samples_epoch//train_batch_size + 1
print("steps_train = {}".format(steps_train))

valid_samples_epoch = valid_generator.samples
print("samples_valid_epoch = {}".format(valid_samples_epoch))

steps_valid = valid_samples_epoch//valid_batch_size + 1
print("steps_valid = {}".format(steps_valid))

samples_train_epoch = 1691
steps_train = 36
samples_valid_epoch = 188
steps_valid = 95


# 3 搭建模型

In [7]:
model = coremodel(basemodel_name, model_image_size)

In [8]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 378, 504, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 188, 251, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 188, 251, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 188, 251, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [9]:
model = multi_gpu_model(model, gpus=2)

# 4 训练模型

In [10]:
# modify myloss, use pure version
model.compile(optimizer=Adam(lr=1e-3), 
              loss=myloss(diff_weight=0, count_weight=1),
              metrics=[count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/50
36/36 [==============================] - 50s 1s/step - loss: 4.3210 - _count_accuracy: 0.0168 - val_loss: 90.6094 - val__count_accuracy: 0.0000e+00
Epoch 2/50
36/36 [==============================] - 36s 1s/step - loss: 1.8394 - _count_accuracy: 0.0630 - val_loss: 9.4333 - val__count_accuracy: 0.0000e+00
Epoch 3/50
36/36 [==============================] - 36s 1s/step - loss: 1.2799 - _count_accuracy: 0.1063 - val_loss: 2.2289 - val__count_accuracy: 0.0000e+00
Epoch 4/50
36/36 [==============================] - 36s 1s/step - loss: 1.1409 - _count_accuracy: 0.1462 - val_loss: 2.2107 - val__count_accuracy: 0.0158
Epoch 5/50
36/36 [==============================] - 36s 1s/step - loss: 0.7752 - _count_accuracy: 0.2368 - val_loss: 0.8643 - val__count_accuracy: 0.0632
Epoch 6/50
36/36 [==============================] - 37s 1s/step - loss: 0.6315 - _count_accuracy: 0.3243 - val_loss: 1.3351 - val__count_accuracy: 0.0421
Epoch 7/50
36/36 [==============================] - 36s 1s/step

In [12]:
model.save_weights("savemodel/"+modelname+"-1.h5")
print("model saved!")

model saved!


In [13]:
model.compile(optimizer=Adam(lr=1e-4), 
              loss=myloss(diff_weight=0, count_weight=1),
              metrics=[count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=50,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/50
36/36 [==============================] - 49s 1s/step - loss: 0.0619 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.9855 - val_loss: 0.1125 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.9211
Epoch 2/50
36/36 [==============================] - 36s 1s/step - loss: 0.0478 - _diff_accuracy: 5.7905e-04 - _count_accuracy: 0.9868 - val_loss: 0.1131 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.9211
Epoch 3/50
36/36 [==============================] - 37s 1s/step - loss: 0.0466 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.9925 - val_loss: 0.1142 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.9474
Epoch 4/50
36/36 [==============================] - 37s 1s/step - loss: 0.0456 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.9898 - val_loss: 0.1369 - val__diff_accuracy: 0.0000e+00 - val__count_accuracy: 0.8895
Epoch 5/50
36/36 [==============================] - 37s 1s/step - loss: 0.0431 - _diff_accuracy: 0.0000e+00 - _count_accuracy: 0.9965 - 

KeyboardInterrupt: 

In [16]:
model.save_weights("savemodel/"+modelname+"-2.h5")
print("model saved!")

model saved!


In [18]:
model.compile(optimizer=RMSprop(lr=1e-5), 
              loss=myloss(diff_weight=0, count_weight=1),
              metrics=[count_accuracy()])
model.fit_generator(train_generator, steps_per_epoch=steps_train, epochs=20,
                   validation_data=valid_generator, validation_steps=steps_valid)

Epoch 1/20
36/36 [==============================] - 53s 1s/step - loss: 0.0161 - _count_accuracy: 0.9975 - val_loss: 0.0957 - val__count_accuracy: 0.9526
Epoch 2/20
36/36 [==============================] - 36s 1s/step - loss: 0.0162 - _count_accuracy: 0.9994 - val_loss: 0.0928 - val__count_accuracy: 0.9526
Epoch 3/20
36/36 [==============================] - 37s 1s/step - loss: 0.0231 - _count_accuracy: 0.9852 - val_loss: 0.0930 - val__count_accuracy: 0.9526
Epoch 4/20
36/36 [==============================] - 38s 1s/step - loss: 0.0161 - _count_accuracy: 1.0000 - val_loss: 0.0920 - val__count_accuracy: 0.9579
Epoch 5/20
36/36 [==============================] - 37s 1s/step - loss: 0.0157 - _count_accuracy: 1.0000 - val_loss: 0.0931 - val__count_accuracy: 0.9526
Epoch 6/20
36/36 [==============================] - 38s 1s/step - loss: 0.0155 - _count_accuracy: 1.0000 - val_loss: 0.0914 - val__count_accuracy: 0.9526
Epoch 7/20
36/36 [==============================] - 38s 1s/step - loss: 0.01

KeyboardInterrupt: 

In [19]:
model.save_weights("savemodel/"+modelname+"-3.h5")
print("model saved!")

model saved!


# 5 测试模型

In [1]:
import os
import cv2
import glob
import numpy as np
import pandas as pd

# seed
np.random.seed(2018)

from keras.optimizers import *
from model.coremodel import coremodel
from dataload.my_image import MyImageDataGenerator
from loss import myloss
from metric import diff_accuracy, count_accuracy

import tensorflow as tf
from keras.models import load_model
print("import done")

# test
basedir = "/home/ubuntu/JupyterNotebook/columncount/data"
basemodel_name = 'inception_v3'
model_image_size = (480, 640)
# define prerpocess_input
from keras.applications.xception import preprocess_input as preprocess_input_ception
from keras.applications.resnet50 import preprocess_input as preprocess_input_resnet
if basemodel_name == 'resnet50':
    preprocess_input = preprocess_input_resnet
elif basemodel_name == 'xception' or 'inception_v3' or 'inception_resnet_v2':
    preprocess_input = preprocess_input_ception
else:
    print('basemodel_name not defined！')
    raise NameError


modelname = basemodel_name + '_20180122'

model = coremodel(basemodel_name, model_image_size)
model.load_weights("savemodel/"+modelname+".h5")

Using TensorFlow backend.


import done
steps_train = 21
steps_valid = 11
Found 20 images.
Found 20 images.
Found 20 images.


In [20]:
gen = MyImageDataGenerator(preprocessing_function=preprocess_input)
test_batch_size = 1
test_generator = gen.myflow_from_directory(os.path.join(basedir,"valid"),  model_image_size,
                                         shuffle=False, batch_size=test_batch_size)
steps_test = test_generator.samples//test_batch_size + 1

Found 188 images.


In [21]:
from postprocess import postprocess

In [24]:
# only count
for i in range(test_generator.samples):
    x, y = test_generator.next()
    pred = model.predict(x)
    out = postprocess(pred[0])
    y2 = y[0][7:].astype(int)
    p2 = out[1]
    flag = (y2 == p2).all()
    if not flag:
        print(test_generator.filenames[i])
        print(y2)
        print(p2)

zhuti2_153_07-51-34.jpg
[4 5 6 6 7 6 6]
[4 5 6 6 7 8 6]
zhuti2_633_02-05-45.jpg
[1 1 1 2 1 2 1]
[1 1 1 2 1 2 2]
zhuti2_850_06-59-27.jpg
[5 5 5 5 5 5 5]
[5 5 6 5 5 5 5]
zhuti2_865_05-50-19.jpg
[2 4 2 2 2 2 2]
[3 4 2 2 2 2 2]
zhuti5_1017_09-31-21-YB.jpg
[2 2 2 2 2 2 2]
[2 2 2 3 2 2 2]
zhuti5_249_05-55-08-YB.jpg
[3 3 3 3 3 3 3]
[3 3 3 3 2 3 3]
zhuti5_540_06-42-51-YB.jpg
[2 2 2 2 0 2 2]
[2 2 2 1 2 2 2]
zhuti5_840_02-00-15.jpg
[4 6 3 4 3 3 3]
[4 6 3 5 3 3 3]


In [ ]:
# include diff and count
for i in range(test_generator.samples):
    x, y = test_generator.next()
    pred = model.predict(x)
    out = postprocess(pred[0])
    print(test_generator.filenames[i])
    y1 = y[0][:7].astype(int)
    print(y1)
    p1 = out[0]
    print(p1)
    y2 = y[0][7:].astype(int)
    print(y2)
    p2 = out[1]
    print(p2)
    flag = ( (y1 == p1).all() and (y2 == p2).all())
    print(flag)